## XORai Demo ##

## Clover Analytics


**Importing Relevant Libraries**

In [4]:
import requests
import json, csv
from random import randint
import sys
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#libraries for JSON parsing function
from functools import (partial,
                       singledispatch)
from itertools import chain
from typing import (Dict,
                    List,
                    TypeVar)

**Constants**

In [6]:
muid = "G6ZYXRGX6VPY1"
mid = "RCTST0000008099"
api_token = "d2dc3c50-9183-8f6f-e688-c1343e02eadd"
environment = "https://sandbox.dev.clover.com/"

**Getting all Order details**

**Steps involved:**
- Connecting to the POS system
- Fetching the order data in JSON format via REST api

In [7]:
endpoint1 = "v3/merchants/"+muid+"/orders?expand=lineItems&expand=payment&expand=lineItems.modifications&expand=serviceCharge&expand=customers"

In [9]:
url = environment + endpoint1 
headers = {"Authorization": "Bearer " + api_token}

response = requests.get(url, headers=headers)

if (response.status_code != 200):
    print(str(response.text.encode('utf8'))+"Something went wrong fetching this merchant's orders")
    sys.exit()
paid = json.loads(response.content)
#print(elements)

order_c = json.dumps(paid,indent = 1)

##dumping all JSON response data to a file
with open(r'/Users/jharnadohotia/Desktop/SFR AI ML/clover_all_orders.json', 'w', encoding='utf-8') as f:
    json.dump(paid, f, ensure_ascii=False, indent=4)

**Function to parse and flatten JSON**

This function is used to parse the JSON data to a python recognised data format, in our case we convert it to a list and then to a Pandas DataFrame

In [10]:
Serializable = TypeVar('Serializable', None, int, bool, float, str, 
                       dict, list, tuple)
Array = List[Serializable]
Object = Dict[str, Serializable]


def flatten(object_: Object,
            *,
            path_separator: str = '.') -> Array[Object]:
    """
    Flattens given JSON object into list of objects with non-nested values.

    >>> flatten({'a': 1})
    [{'a': 1}]
    >>> flatten({'a': [1, 2]})
    [{'a': 1}, {'a': 2}]
    >>> flatten({'a': {'b': None}})
    [{'a.b': None}]
    """
    keys = set(object_)
    result = [dict(object_)]
    while keys:
        key = keys.pop()
        new_result = []
        for index, record in enumerate(result):
            try:
                value = record[key]
            except KeyError:
                new_result.append(record)
            else:
                if isinstance(value, dict):
                    del record[key]
                    new_value = flatten_nested_objects(
                            value,
                            prefix=key + path_separator,
                            path_separator=path_separator)
                    keys.update(new_value.keys())
                    new_result.append({**new_value, **record})
                elif isinstance(value, list):
                    del record[key]
                    new_records = [
                        flatten_nested_objects(sub_value,
                                               prefix=key + path_separator,
                                               path_separator=path_separator)
                        for sub_value in value]
                    keys.update(chain.from_iterable(map(dict.keys,
                                                        new_records)))
                    new_result.extend({**new_record, **record}
                                      for new_record in new_records)
                else:
                    new_result.append(record)
        result = new_result
    return result


@singledispatch
def flatten_nested_objects(object_: Serializable,
                           *,
                           prefix: str = '',
                           path_separator: str) -> Object:
    return {prefix[:-len(path_separator)]: object_}


@flatten_nested_objects.register(dict)
def _(object_: Object,
      *,
      prefix: str = '',
      path_separator: str) -> Object:
    result = dict(object_)
    for key in list(result):
        result.update(flatten_nested_objects(result.pop(key),
                                             prefix=(prefix + key
                                                     + path_separator),
                                             path_separator=path_separator))
    return result


@flatten_nested_objects.register(list)
def _(object_: Array,
      *,
      prefix: str = '',
      path_separator: str) -> Object:
    return {prefix[:-len(path_separator)]: list(map(partial(
            flatten_nested_objects,
            path_separator=path_separator),
            object_))}

In [11]:
new = flatten(paid)
#print(type(new))

<class 'list'>


**Orders Dataframe**

Creating a python dataframe to load all the order data and process it

In [55]:
dfr = pd.DataFrame.from_dict(new)

In [56]:
dfr.head()

,elements.lineItems.elements.id,elements.lineItems.elements.orderRef.id,elements.lineItems.elements.item.id,elements.lineItems.elements.name,elements.lineItems.elements.alternateName,elements.lineItems.elements.price,elements.lineItems.elements.unitQty,elements.lineItems.elements.unitName,elements.lineItems.elements.itemCode,elements.lineItems.elements.printed,...,elements.payments.elements.tender.href,elements.payments.elements.tender.id,elements.payments.elements.amount,elements.payments.elements.taxAmount,elements.payments.elements.employee.id,elements.payments.elements.createdTime,elements.payments.elements.clientCreatedTime,elements.payments.elements.modifiedTime,elements.payments.elements.offline,elements.payments.elements.result
0,178AG1FVGFDRE,XCK89FJW51CDM,DB9NSDPZPTWVE,Smoothie,ProductName1,1000,1,1,1234500001,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1XEZ5CTBBPT54,XCK89FJW51CDM,4WWNAX1TD8G4Y,Instant coffee,ProductName8,1000,1,1,1234500008,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B4MWH2AJFNMAA,A8FTY2RC86C1W,4WWNAX1TD8G4Y,Instant coffee,ProductName8,1000,1,1,1234500008,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0639W75D2W8QW,0ERWZMJTP48TJ,J2VQ6VWD2BY62,Soda,ProductName5,1000,1,1,1234500005,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M75EG7Z6KT5QM,0ERWZMJTP48TJ,10847C9VKVNC6,Tea,ProductName9,1000,1,1,1234500009,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
